# CLIP

In [4]:
#!pip3 install clip-by-openai
!pip install git+https://github.com/openai/CLIP.git

Looking in indexes: https://pypi.org/simple, https://pypi.ngc.nvidia.com
  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-qpm5or2f
  Running command git clone -q https://github.com/openai/CLIP.git /tmp/pip-req-build-qpm5or2f
  Resolved https://github.com/openai/CLIP.git to commit b46f5ac7587d2e1862f8b7b1573179d80dcdd620
     |████████████████████████████████| 53 kB 4.4 MB/s eta 0:00:011
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369387 sha256=7b3c00aaaaf9ab2428492f9f8dbc72ca46fe840b1a89f7f53870de3a4768e0d9
  Stored in directory: /tmp/pip-ephem-wheel-cache-y0ua70lx/wheels/ab/4f/3a/5e51521b55997aa6f0690e095c08824219753128ce8d9969a3
Successfully built clip


In [7]:
import torch.backends.cudnn as cudnn
import torch
import torchvision.transforms as transforms
import PIL
import argparse
import os
import random
import sys
import pprint
import datetime
import dateutil.tz
import numpy as np
import json
import functools

In [119]:
import torch
import clip
from PIL import Image

## Pre-Trained Model

In [120]:
device = "cuda:0" if torch.cuda.is_available() else "cpu" # If using GPU then use mixed precision training.
model, preprocess = clip.load("ViT-B/32",device=device,jit=False) #Must set jit=False for training

/opt/conda/lib/python3.8/site-packages/torch/nn/modules/module.py:1383: UserWarning: positional arguments and argument "destination" are deprecated. nn.Module.state_dict will not accept them in the future. Refer to https://pytorch.org/docs/master/generated/torch.nn.Module.html#torch.nn.Module.state_dict for details.
  warnings.warn(


In [149]:
from torch.utils.data import Dataset, DataLoader

# --------------------------------------------- #
#                  Data Utils
# --------------------------------------------- #

import os, pickle, re, csv
from tqdm import tqdm
import numpy as np
import torch.utils.data
from torchvision.datasets import ImageFolder
from torch.utils.data import Dataset, DataLoader
import torchvision.transforms as transforms
import PIL
from collections import Counter
import nltk
import json
import functools

image_transforms = transforms.Compose([PIL.Image.fromarray, 
                                           transforms.Resize((64, 64)),
                                           #transforms.RandomHorizontalFlip(),
                                           transforms.ToTensor(),
                                           transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
        # dataset = TextDataset(cfg.DATA_DIR, 'train',
        #                       imsize=cfg.IMSIZE,
        #                       transform=image_transform)
        #assert dataset
def video_transform(video, image_transform):
    vid = []
    for im in video:
        vid.append(image_transform(im))
    vid = torch.stack(vid).permute(1, 0, 2, 3)
    print("vid value: ", vid)
    return vid

video_len = 5
n_channels = 3
    # functools.partial takes methods/functions as an input
video_transforms = functools.partial(video_transform, image_transform=image_transforms)

class VideoFolderDataset(torch.utils.data.Dataset):
    def __init__(self, folder, counter = None, cache=None, min_len=4, mode='train', load_images=False, id_file='train_seen_unseen_ids.npy'):
        self.lengths = []
        self.followings = []
        dataset = ImageFolder(folder)
        self.dir_path = folder
        self.total_frames = 0
        self.images = []
        self.labels = np.load(os.path.join(folder, 'labels.npy'), allow_pickle=True, encoding='latin1').item()
        if cache is not None and os.path.exists(cache + 'img_cache' + str(min_len) + '.npy') and os.path.exists(cache + 'following_cache' + str(min_len) +  '.npy'):
            self.images = np.load(cache + 'img_cache' + str(min_len) + '.npy', encoding='latin1')
            self.followings = np.load(cache + 'following_cache' + str(min_len) + '.npy')
        else:
            for idx, (im, _) in enumerate(
                    tqdm(dataset, desc="Counting total number of frames")):
                img_path, _ = dataset.imgs[idx]
                v_name = img_path.replace(folder,'')
                id = v_name.split('/')[-1]
                id = int(id.replace('.png', ''))
                v_name = re.sub(r"[0-9]+.png",'', v_name)
                if id > counter[v_name] - min_len:
                    continue
                following_imgs = []
                for i in range(min_len):
                    following_imgs.append(v_name + str(id+i+1) + '.png')
                self.images.append(img_path.replace(folder, ''))
                self.followings.append(following_imgs)
            np.save(os.path.join(folder, 'img_cache' + str(min_len) + '.npy'), self.images)
            np.save(os.path.join(folder, 'following_cache' + str(min_len) + '.npy'), self.followings)

        # train_id, test_id = np.load(self.dir_path + 'train_test_ids.npy', allow_pickle=True, encoding='latin1')
        train_id, val_id, test_id = np.load(os.path.join(self.dir_path, id_file), allow_pickle=True)
        if mode == 'train':
            orders = train_id
        elif mode =='val':
            orders = val_id[:2320]
        elif mode == 'test':
            orders = test_id
        else:
            raise ValueError
        orders = np.array(orders).astype('int32')
        self.images = self.images[orders]
        self.followings = self.followings[orders]
        print("Total number of clips {}".format(len(self.images)))

        self.image_arrays = {}
        if load_images:
            for idx, (im, _) in enumerate(
                    tqdm(dataset, desc="Counting total number of frames")):
                img_path, _ = dataset.imgs[idx]
                self.image_arrays[img_path] = im

    def sample_image(self, im):
        shorter, longer = min(im.size[0], im.size[1]), max(im.size[0], im.size[1])
        video_len = int(longer/shorter)
        se = np.random.randint(0, video_len, 1)[0]
        print(se*shorter, shorter, (se+1)*shorter)
        return im.crop((0, se * shorter, shorter, (se+1)*shorter)), se

    def __getitem__(self, item):
        lists = [self.images[item]]
        for i in range(len(self.followings[item])):
            lists.append(str(self.followings[item][i]))
        return lists

    def __len__(self):
        return len(self.images)

    
class ImageDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, textvec, transform):
        self.dir_path = dataset.dir_path
        self.dataset = dataset
        self.transforms = transform
        self.descriptions = np.load(textvec + 'descriptions_vec.npy', allow_pickle=True, encoding='latin1').item()
        self.attributes =  np.load(textvec + 'descriptions_attr.npy', allow_pickle=True, encoding='latin1').item()
        self.subtitles = np.load(textvec + 'subtitles_vec.npy', allow_pickle=True, encoding='latin1').item()
        self.descriptions_original = np.load(textvec + 'descriptions.npy', allow_pickle=True, encoding='latin1').item()
        self.labels = dataset.labels

    def __getitem__(self, item):
        path = self.dir_path + str(self.dataset[item][0])[2:-1]
        id = str(self.dataset[item][0]).replace('.png','')[2:-1]
        img_id = id
        im = PIL.Image.open(path)
        image, sample_idx = self.dataset.sample_image(im)
        image = self.transforms(np.array(image))
        subs = self.subtitles[id][0]
        se = 0
        if len(self.descriptions_original[id]) > 1:
            se = np.random.randint(0,len(self.descriptions_original[id]),1)
            se = se[0]
        des = self.descriptions[id][se]
        attri = self.attributes[id][se].astype('float32')
        text = self.descriptions_original[id][se]
        label = self.labels[id].astype(np.float32)
        input_id = None
        mask = None
        if self.return_caption:
            input_id, mask = self._sentence_to_idx(self.tokenized_descriptions[id][se])
            input_id = np.array(input_id)
            mask = np.array(mask)

        lists = self.dataset[item]
        content = []
        attri_content = []
        attri_label = []
        for v in lists:
            id =str(v).replace('.png','')[2:-1]
            se = 0
            if len(self.descriptions[id]) > 1:
                se = np.random.randint(0,len(self.descriptions[id]),1)
                se = se[0]
            content.append(np.expand_dims(self.descriptions[id][se], axis = 0))
            attri_content.append(np.expand_dims(self.attributes[id][se].astype('float32'), axis = 0))
            attri_label.append(np.expand_dims(self.labels[id].astype('float32'), axis = 0))
        content = np.concatenate(content, axis = 0)
        attri_content = np.concatenate(attri_content, axis = 0)
        attri_label = np.concatenate(attri_label, axis = 0)
        content = np.concatenate([content, attri_content, attri_label], 1)
        des = np.concatenate([des, attri])
        ##
        content = torch.tensor(content)

        data_item = {'images': image, 'text':text, 'description': des,
                'subtitle': subs, 'labels':label, 'content': content}
        
        image_item = image

        if self.return_caption:
            input_id = torch.tensor(input_id)
            mask = torch.tensor(mask)
            data_item.update({'input_id': input_id, 'mask':mask})
        return image_item
        #return data_item

    def __len__(self):
        return len(self.dataset.images)

In [142]:
import os, pickle
from tqdm import tqdm
import numpy as np
import torch.utils.data
import PIL
from random import randrange
from collections import Counter
import nltk
import json
import torchvision.transforms as transforms
from storygen.config import cfg, cfg_from_file

image_transforms = transforms.Compose([PIL.Image.fromarray, 
                                           transforms.Resize((cfg.IMSIZE, cfg.IMSIZE)),
                                           #transforms.RandomHorizontalFlip(),
                                           transforms.ToTensor(),
                                           transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
        # dataset = TextDataset(cfg.DATA_DIR, 'train',
        #                       imsize=cfg.IMSIZE,
        #                       transform=image_transform)
        #assert dataset
def video_transform(video, image_transform):
    vid = []
    for im in video:
        vid.append(image_transform(im))
    vid = torch.stack(vid).permute(1, 0, 2, 3)
    print("vid value: ", vid)
    return vid

video_len = 5
n_channels = 3
# functools.partial takes methods/functions as an input
video_transforms = functools.partial(video_transform, image_transform=image_transforms)
    
    

unique_characters = ["Wilma", "Fred", "Betty", "Barney", "Dino", "Pebbles", "Mr Slate"]
class VideoFolderDataset(torch.utils.data.Dataset):
    def __init__(self, folder, cache=None, min_len=1, mode='train'):
        self.lengths = []
        self.followings = {}
        self.dir_path = folder
        self.total_frames = 0

        # train_id, test_id = np.load(self.dir_path + 'train_test_ids.npy', allow_pickle=True, encoding='latin1')
        splits = json.load(open(os.path.join(self.dir_path, 'train-val-test_split.json'), 'r'))
        train_id, val_id, test_id = splits["train"], splits["val"], splits["test"]

        if os.path.exists(cache + 'following_cache' + str(min_len) +  '.npy'):
            self.followings = pickle.load(open(cache + 'following_cache' + str(min_len) + '.pkl', 'rb'))
        else:
            all_clips = train_id + val_id + test_id
            all_clips.sort()
            for idx, clip in enumerate(all_clips):
                season, episode = int(clip.split('_')[1]), int(clip.split('_')[3])
                has_frames = True
                for c in all_clips[idx+1:idx+min_len+1]:
                    s_c, e_c = int(c.split('_')[1]), int(c.split('_')[3])
                    if s_c != season or e_c != episode:
                        has_frames = False
                        break
                if has_frames:
                    self.followings[clip] = all_clips[idx+1:idx+min_len+1]
                else:
                    continue
            pickle.dump(self.followings, open(os.path.join(folder, 'following_cache' + str(min_len) + '.pkl'), 'wb'))



        if os.path.exists(os.path.join(folder, 'labels.pkl')):
            self.labels = pickle.load(open(os.path.join(folder, 'labels.pkl'), 'rb'))
        else:
            print("Computing and saving labels")
            annotations = json.load(open(os.path.join(folder, 'flintstones_annotations_v1-0.json'), 'r'))
            self.labels = {}
            for sample in annotations:
                sample_characters = [c["entityLabel"].strip().lower() for c in sample["characters"]]
                self.labels[sample["globalID"]] = [1 if c.lower() in sample_characters else 0 for c in unique_characters]
            pickle.dump(self.labels, open(os.path.join(folder, 'labels.pkl'), 'wb'))

        self.embeds = np.load(os.path.join(self.dir_path, "flintstones_use_embeddings.npy"))
        self.sent2idx = pickle.load(open(os.path.join(self.dir_path, 'flintstones_use_embed_idxs.pkl'), 'rb'))

        self.filtered_followings = {}
        for i, f in self.followings.items():
            #print(f)
            if len(f) == 4:
                self.filtered_followings[i] = f
            else:
                continue
        self.followings = self.filtered_followings

        train_id = [tid for tid in train_id if tid in self.followings]
        val_id = [vid for vid in val_id if vid in self.followings]
        test_id = [tid for tid in test_id if tid in self.followings]

        if mode == 'train':
            self.orders = train_id
        elif mode =='val':
            self.orders = val_id
        elif mode == 'test':
            self.orders = test_id
        else:
            raise ValueError
        print("Total number of clips {}".format(len(self.orders)))

    def sample_image(self, im):
        shorter, longer = min(im.size[0], im.size[1]), max(im.size[0], im.size[1])
        video_len = int(longer/shorter)
        se = np.random.randint(0, video_len, 1)[0]
        #print(se*shorter, shorter, (se+1)*shorter)
        return im.crop((0, se * shorter, shorter, (se+1)*shorter)), se

    def __getitem__(self, item):
        return [self.orders[item]] + self.followings[self.orders[item]]

    def __len__(self):
        return len(self.orders)


class StoryDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, transform, return_caption=False, out_dir=None, densecap=False):
        self.dir_path = dataset.dir_path
        self.dataset = dataset
        self.transforms = transform
        self.labels = dataset.labels
        self.return_caption = return_caption

        annotations = json.load(open(os.path.join(self.dir_path, 'flintstones_annotations_v1-0.json')))
        self.descriptions = {}
        for sample in annotations:
            self.descriptions[sample["globalID"]] = sample["description"]

        if self.return_caption:
            self.init_mart_vocab()
            self.max_len = self.tokenize_descriptions()
            print("Max sequence length = %s" % self.max_len)
        else:
            self.vocab = None
        self.out_dir = out_dir

        # if densecap:
        #     self.densecap_dataset = DenseCapDataset(self.dir_path)
        # else:
        self.densecap_dataset = None

    def tokenize_descriptions(self):
        caption_lengths = []
        self.tokenized_descriptions = {}
        for img_id, descs in self.descriptions.items():
            self.tokenized_descriptions[img_id] = nltk.tokenize.word_tokenize(descs.lower())
            caption_lengths.append(len(self.tokenized_descriptions[img_id]))
        return max(caption_lengths) + 2

    def init_mart_vocab(self):

        vocab_file = os.path.join(self.dir_path, 'mart_vocab.pkl')
        if os.path.exists(vocab_file):
            vocab_from_file = True
        else:
            vocab_from_file = False

        self.vocab = Vocabulary(vocab_threshold=5,
                                vocab_file=vocab_file,
                                annotations_file=os.path.join(self.dir_path, 'flintstones_annotations_v1-0.json'),
                                vocab_from_file=vocab_from_file)

    def save_story(self, output, save_path = './'):
        all_image = []
        images = output['images_numpy']
        texts = output['text']
        for i in range(images.shape[0]):
            all_image.append(np.squeeze(images[i]))
        output = PIL.Image.fromarray(np.concatenate(all_image, axis = 0))
        output.save(save_path + 'image.png')
        fid = open(save_path + 'text.txt', 'w')
        for i in range(len(texts)):
            fid.write(texts[i] +'\n' )
        fid.close()
        return

    def _sentence_to_idx(self, sentence_tokens):
        """[BOS], [WORD1], [WORD2], ..., [WORDN], [EOS], [PAD], ..., [PAD], len == max_t_len
        All non-PAD values are valid, with a mask value of 1
        """
        max_t_len = self.max_len
        sentence_tokens = sentence_tokens[:max_t_len - 2]

        # pad
        valid_l = len(sentence_tokens)
        mask = [1] * valid_l + [0] * (max_t_len - valid_l)
        sentence_tokens += [self.vocab.pad_word] * (max_t_len - valid_l)
        input_ids = [self.vocab.word2idx.get(t, self.vocab.word2idx[self.vocab.unk_word]) for t in sentence_tokens]

        return input_ids, mask

    def __getitem__(self, item):
        lists = self.dataset[item]
        labels = []
        images = []
        text = []
        input_ids = []
        masks= []
        sent_embeds = []
        for idx, globalID in enumerate(lists):
            if self.out_dir:
                im = PIL.Image.open(os.path.join(self.out_dir, 'img-%s-%s.png' % (item, idx))).convert('RGB')
            else:
                arr = np.load(os.path.join(self.dir_path, 'video_frames_sampled', globalID + '.npy'))
                n_frames = arr.shape[0]
                im = arr[randrange(n_frames)]
            images.append(np.expand_dims(np.array(im), axis=0))
            text.append(self.descriptions[globalID])
            labels.append(np.expand_dims(self.labels[globalID], axis = 0))
            sent_embeds.append(np.expand_dims(self.dataset.embeds[self.dataset.sent2idx[globalID]], axis = 0))

            if self.return_caption:
                input_id, mask = self._sentence_to_idx(self.tokenized_descriptions[globalID])
                input_ids.append(np.expand_dims(input_id, axis=0))
                masks.append(np.expand_dims(mask, axis=0))

        sent_embeds = np.concatenate(sent_embeds, axis = 0)
        labels = np.concatenate(labels, axis = 0)
        images = np.concatenate(images, axis = 0)
        # image is T x H x W x C
        transformed_images = self.transforms(images)
        # After transform, image is C x T x H x W

        sent_embeds = torch.tensor(sent_embeds)
        labels = torch.tensor(np.array(labels).astype(np.float32))

        data_item = {'images': transformed_images, 'text':text, 'description': sent_embeds, 'images_numpy':images, 'labels':labels}

        if self.return_caption:
            input_ids = torch.tensor(np.concatenate(input_ids))
            masks = torch.tensor(np.concatenate(masks))
            data_item.update({'input_ids': input_ids, 'masks': masks})

        if self.densecap_dataset:
            boxes, caps, caps_len = [], [], []
            for idx, v in enumerate(lists):
                img_id = str(v).replace('.png', '')[2:-1]
                path = img_id + '.png'
                boxes.append(torch.as_tensor([ann['box'] for ann in self.densecap_dataset[path]], dtype=torch.float32))
                caps.append(torch.as_tensor([ann['cap_idx'] for ann in self.densecap_dataset[path]], dtype=torch.long))
                caps_len.append(torch.as_tensor([sum([1 for k in ann['cap_idx'] if k!= 0]) for ann in self.densecap_dataset[path]], dtype=torch.long))
            targets = {
                'boxes': torch.cat(boxes),
                'caps': torch.cat(caps),
                'caps_len': torch.cat(caps_len),
            }
            data_item.update(targets)

        return data_item

    def __len__(self):
        return len(self.dataset.orders)


class ImageDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, transform, return_caption=False, densecap=False):
        self.dir_path = dataset.dir_path
        self.dataset = dataset
        self.transforms = transform
        self.labels = dataset.labels
        self.return_caption = return_caption

        annotations = json.load(open(os.path.join(self.dir_path, 'flintstones_annotations_v1-0.json')))
        self.descriptions = {}
        for sample in annotations:
            self.descriptions[sample["globalID"]] = sample["description"]

        if self.return_caption:
            self.init_mart_vocab()
            self.max_len = self.tokenize_descriptions()
            print("Max sequence length = %s" % self.max_len)
        else:
            self.vocab = None

        # if densecap:
        #     self.densecap_dataset = DenseCapDataset(self.dir_path)
        # else:
        self.densecap_dataset = None

    def tokenize_descriptions(self):
        caption_lengths = []
        self.tokenized_descriptions = {}
        for img_id, descs in self.descriptions.items():
            self.tokenized_descriptions[img_id] = nltk.tokenize.word_tokenize(descs.lower())
            caption_lengths.append(len(self.tokenized_descriptions[img_id]))
        return max(caption_lengths) + 2

    def _sentence_to_idx(self, sentence_tokens):
        """[BOS], [WORD1], [WORD2], ..., [WORDN], [EOS], [PAD], ..., [PAD], len == max_t_len
        All non-PAD values are valid, with a mask value of 1
        """
        max_t_len = self.max_len
        sentence_tokens = sentence_tokens[:max_t_len - 2]

        # pad
        valid_l = len(sentence_tokens)
        mask = [1] * valid_l + [0] * (max_t_len - valid_l)
        sentence_tokens += [self.vocab.pad_word] * (max_t_len - valid_l)
        input_ids = [self.vocab.word2idx.get(t, self.vocab.word2idx[self.vocab.unk_word]) for t in sentence_tokens]

        return input_ids, mask

    def init_mart_vocab(self):

        vocab_file = os.path.join(self.dir_path, 'mart_vocab.pkl')
        if os.path.exists(vocab_file):
            vocab_from_file = True
        else:
            vocab_from_file = False

        self.vocab = Vocabulary(vocab_threshold=5,
                                vocab_file=vocab_file,
                                annotations_file=os.path.join(self.dir_path, 'flintstones_annotations_v1-0.json'),
                                vocab_from_file=vocab_from_file)

    def __getitem__(self, item):

        # single image input
        globalID = self.dataset[item][0]
        arr = np.load(os.path.join(self.dir_path, 'video_frames_sampled', globalID + '.npy'))
        n_frames = arr.shape[0]
        print("n_frames: %d" % n_frames)
        im = arr[randrange(n_frames)]
        #print("img")
        #print(im)
        image = np.array(im)
        #PIL_image = Image.fromarray(image)
        #image = preprocess(Image.open(PIL_image))
        image = preprocess(Image.fromarray(image))
        print(image.shape)
        
        text = self.descriptions[globalID]
        label = np.array(self.labels[globalID]).astype(np.float32)
        sent_embed = self.dataset.embeds[self.dataset.sent2idx[globalID]]

        input_id = None
        mask = None
        if self.return_caption:
            input_id, mask = self._sentence_to_idx(self.tokenized_descriptions[globalID])
            input_id = np.array(input_id)
            mask = np.array(mask)

        # input ofr conditional vector
        lists = self.dataset[item]
        sent_embeds = []
        for idx, globalID in enumerate(lists):
            sent_embeds.append(np.expand_dims(self.dataset.embeds[self.dataset.sent2idx[globalID]], axis=0))
        sent_embeds = np.concatenate(sent_embeds, axis=0)

        ##
        sent_embeds = torch.tensor(sent_embeds)
        #image = self.transforms(image)
        data_item = {'images': image, 'text':text, 'description': sent_embed,
                     'labels':label, 'content': sent_embeds}

        if self.return_caption:
            input_id = torch.tensor(input_id)
            mask = torch.tensor(mask)
            data_item.update({'input_id': input_id, 'mask':mask})

        if self.densecap_dataset:
            path = globalID + '.png'
            try:
                _ = self.densecap_dataset[path]
            except KeyError:
                shorter, longer = min(im.size[0], im.size[1]), max(im.size[0], im.size[1])
                video_len = int(longer / shorter)
                raise KeyError

            boxes = torch.as_tensor([ann['box'] for ann in self.densecap_dataset[path]], dtype=torch.float32)
            caps = torch.as_tensor([ann['cap_idx'] for ann in self.densecap_dataset[path]], dtype=torch.long)
            caps_len = torch.as_tensor([sum([1 for k in ann['cap_idx'] if k!= 0]) for ann in self.densecap_dataset[path]], dtype=torch.long)
            targets = {
                'boxes': boxes,
                'caps': caps,
                'caps_len': caps_len,
            }
            data_item.update(targets)

        #return data_item
        return image, text

    def __len__(self):
        return len(self.dataset.orders)


class StoryImageDataset(torch.utils.data.Dataset):
    def __init__(self, data_folder, im_input_size,
                 out_img_folder = None,
                 mode='train',
                 video_len = 5,
                 transform=None):
        self.followings = {}
        self.data_folder = data_folder
        self.labels = pickle.load(open(os.path.join(data_folder, 'labels.pkl'), 'rb'))
        self.video_len = video_len
        min_len = video_len-1

        splits = json.load(open(os.path.join(self.data_folder, 'train-val-test_split.json'), 'r'))
        train_ids, val_ids, test_ids = splits["train"], splits["val"], splits["test"]

        if os.path.exists(os.path.join(data_folder, 'following_cache' + str(video_len-1) +  '.pkl')):
            self.followings = pickle.load(open(os.path.join(data_folder, 'following_cache' + str(video_len-1) + '.pkl'), 'rb'))
        else:
            all_clips = train_ids + val_ids + test_ids
            all_clips.sort()
            for idx, clip in enumerate(tqdm(all_clips, desc="Counting total number of frames")):
                season, episode = int(clip.split('_')[1]), int(clip.split('_')[3])
                has_frames = True
                for c in all_clips[idx+1:idx+min_len+1]:
                    s_c, e_c = int(c.split('_')[1]), int(c.split('_')[3])
                    if s_c != season or e_c != episode:
                        has_frames = False
                        break
                if has_frames:
                    self.followings[clip] = all_clips[idx+1:idx+min_len+1]
                else:
                    continue
            pickle.dump(self.followings, open(os.path.join(self.data_folder, 'following_cache' + str(min_len) + '.pkl'), 'wb'))

        self.filtered_followings = {}
        for i, f in self.followings.items():
            #print(f)
            if len(f) == 4:
                self.filtered_followings[i] = f
            else:
                continue
        self.followings = self.filtered_followings

        train_ids = [tid for tid in train_ids if tid in self.followings]
        val_ids = [vid for vid in val_ids if vid in self.followings]
        test_ids = [tid for tid in test_ids if tid in self.followings]

        # print(list(self.followings.keys())[:10])

        if mode == 'train':
            self.ids = train_ids
            self.transform = transforms.Compose([
                # Image.fromarray,
                transforms.Resize(im_input_size),
                transforms.RandomHorizontalFlip(),
                transforms.ToTensor(),
                # transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])
        else:
            self.ids = val_ids[:2060] if mode == "val" else test_ids[:2304]
            self.transform = transforms.Compose([
                # Image.fromarray,
                transforms.Resize(im_input_size),
                transforms.CenterCrop(im_input_size),
                transforms.ToTensor(),
                # transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
            ])


        self.out_dir = out_img_folder

    def __getitem__(self, item):

        globalIDs = [self.ids[item]] + self.followings[self.ids[item]]

        images = []
        for idx, globalID in enumerate(globalIDs):
            if self.out_dir:
                im = PIL.Image.open(os.path.join(self.out_dir, 'img-%s-%s.png' % (item, idx))).convert('RGB')
                images.append(im)
            else:
                arr = np.load(os.path.join(self.data_folder, 'video_frames', globalID + '.npy'))
                n_frames = arr.shape[0]
                im = arr[randrange(n_frames)]
                # images.append(np.expand_dims(np.array(im), axis = 0))
                images.append(PIL.Image.fromarray(im))

        # print([(type(im)) for im in images])

        labels = [self.labels[globalID] for globalID in globalIDs]
        return torch.stack([self.transform(image).squeeze(0) for image in images]), torch.tensor(np.vstack(labels))

    def __len__(self):
        return len(self.ids)

In [143]:
import tqdm
dir_path = "./flintstones_data/"
base = VideoFolderDataset(dir_path, cache = "./flintstones_data/", min_len = 4, mode="train")
imagedataset = ImageDataset(base, image_transforms)
#stimagedataset = StoryImageDataset(base, image_transforms)

Total number of clips 20132


In [144]:
image, text = next(iter(imagedataset))

#from PIL import Image
#import numpy as np

#PIL_image = Image.fromarray(np.uint8(numpy_image)).convert('RGB')

n_frames: 5
torch.Size([3, 224, 224])


In [135]:
image.shape

(5, 128, 128, 3)

In [114]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
#w, h = 512, 512
#data = np.zeros((h, w, 3), dtype=np.uint8)
#data[0:256, 0:256] = [255, 0, 0] # red patch in upper left
#image = image.numpy()
#im = Image.fromarray(( image* 255).astype(np.uint8))
#Image.fromarray(image).show()
#image.show()
#plt.show()

In [115]:
from torch.utils.data import Dataset, DataLoader
def load_data():
    #dir_path = args.dataset_path
    dir_path = "./pororo_data/"
    counter = np.load(os.path.join(dir_path, 'frames_counter.npy'), allow_pickle=True).item()
    print("The number of frames: ", len(counter))
    base = VideoFolderDataset(dir_path, counter = counter, cache = dir_path, min_len = 4, mode="train")
    #storydataset = StoryDataset(base, dir_path, video_transforms)
    #imagedataset = ImageDataset(base, dir_path, image_transforms)
    storydataset = StoryDataset(base, dir_path, video_transforms)
    #train_data = ImagePaths(args.dataset_path, size=256)
    #train_loader = DataLoader(imagedataset, batch_size=6, shuffle=False)
    story_loader = DataLoader(storydataset, batch_size=6, shuffle=False)
    return story_loader

In [122]:
import torch.nn as nn

class image_title_dataset(Dataset):
    def __init__(self, list_image_path,list_txt):

        self.image_path = list_image_path
        self.title  = clip.tokenize(list_txt) #you can tokenize everything at once in here(slow at the beginning), or tokenize it in the training loop.

    def __len__(self):
        return len(list_txt)

    def __getitem__(self, idx):
        image = preprocess(Image.open(self.image_path[idx])) # Image from PIL module
        title = self.title[idx]
        return image, title

# use your own data
list_image_path = ['./1.png'] 
list_txt = ['description for image1.jpg']
dataset = image_title_dataset(list_image_path, list_txt)
train_dataloader = DataLoader(dataset) #Define your own dataloader

In [123]:
im, ti = next(iter(train_dataloader))

In [124]:
im.shape

torch.Size([1, 3, 224, 224])

In [50]:
ti.shape

torch.Size([1, 77])

In [ ]:
import torch.nn as nn
import torch.optim as optim

#https://github.com/openai/CLIP/issues/57
def convert_models_to_fp32(model): 
    for p in model.parameters(): 
        p.data = p.data.float() 
        p.grad.data = p.grad.data.float() 

#if device == "cpu":
#    model.float()
#else :
#    clip.model.convert_weights(model) # Actually this line is unnecessary since clip by default already on float16

loss_img = nn.CrossEntropyLoss()
loss_txt = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=5e-5,betas=(0.9,0.98),eps=1e-6,weight_decay=0.2) #Params used from paper, the lr is smaller, more safe for fine tuning to new dataset

# add your own code to track the training progress.
for epoch in range(10):
    for batch in imagedataset :
        optimizer.zero_grad()
        images, texts = batch
        #new_im = np.dstack((images, np.ones(images.shape[:2])))
        images =  images.unsqueeze(1)
        images = torch.transpose(images, 0, 1)
        print("shape" + str(images.shape))
        #images = preprocess(images)
        texts = clip.tokenize(texts)
        
        images= images.to(device)
        texts = texts.to(device)
        print(images.shape)
        print(texts.shape)
        logits_per_image, logits_per_text = model(images, texts)
        ground_truth = torch.arange(len(images),dtype=torch.long,device=device)
        total_loss = (loss_img(logits_per_image,ground_truth) + loss_txt(logits_per_text,ground_truth))/2
        total_loss.backward()
        if device == "cpu":
            optimizer.step()
        else : 
            convert_models_to_fp32(model)
            optimizer.step()
            clip.model.convert_weights(model)

n_frames: 5
torch.Size([3, 224, 224])
shapetorch.Size([1, 3, 224, 224])
torch.Size([1, 3, 224, 224])
torch.Size([1, 77])
n_frames: 5
torch.Size([3, 224, 224])
shapetorch.Size([1, 3, 224, 224])
torch.Size([1, 3, 224, 224])
torch.Size([1, 77])
n_frames: 5
torch.Size([3, 224, 224])
shapetorch.Size([1, 3, 224, 224])
torch.Size([1, 3, 224, 224])
torch.Size([1, 77])
n_frames: 5
torch.Size([3, 224, 224])
shapetorch.Size([1, 3, 224, 224])
torch.Size([1, 3, 224, 224])
torch.Size([1, 77])
n_frames: 5
torch.Size([3, 224, 224])
shapetorch.Size([1, 3, 224, 224])
torch.Size([1, 3, 224, 224])
torch.Size([1, 77])
n_frames: 5
torch.Size([3, 224, 224])
shapetorch.Size([1, 3, 224, 224])
torch.Size([1, 3, 224, 224])
torch.Size([1, 77])
n_frames: 5
torch.Size([3, 224, 224])
shapetorch.Size([1, 3, 224, 224])
torch.Size([1, 3, 224, 224])
torch.Size([1, 77])
n_frames: 5
torch.Size([3, 224, 224])
shapetorch.Size([1, 3, 224, 224])
torch.Size([1, 3, 224, 224])
torch.Size([1, 77])
n_frames: 5
torch.Size([3, 224, 

In [121]:
for epoch in range(10):
    for batch in train_dataloader :
        optimizer.zero_grad()
        images,texts = batch
        print(len(batch))
        images= images.to(device)
        texts = texts.to(device)
        print(images.shape)
        print(texts.shape)
        logits_per_image, logits_per_text = model(images, texts)
        ground_truth = torch.arange(len(images),dtype=torch.long,device=device)
        total_loss = (loss_img(logits_per_image,ground_truth) + loss_txt(logits_per_text,ground_truth))/2
        total_loss.backward()
        if device == "cpu":
            optimizer.step()
        else : 
            convert_models_to_fp32(model)
            optimizer.step()
            clip.model.convert_weights(model)

2
torch.Size([1, 3, 224, 224])
torch.Size([1, 77])
2
torch.Size([1, 3, 224, 224])
torch.Size([1, 77])
2
torch.Size([1, 3, 224, 224])
torch.Size([1, 77])
2
torch.Size([1, 3, 224, 224])
torch.Size([1, 77])
2
torch.Size([1, 3, 224, 224])
torch.Size([1, 77])
2
torch.Size([1, 3, 224, 224])
torch.Size([1, 77])
2
torch.Size([1, 3, 224, 224])
torch.Size([1, 77])
2
torch.Size([1, 3, 224, 224])
torch.Size([1, 77])
2
torch.Size([1, 3, 224, 224])
torch.Size([1, 77])
2
torch.Size([1, 3, 224, 224])
torch.Size([1, 77])


In [100]:
model

CLIP(
  (visual): VisionTransformer(
    (conv1): Conv2d(3, 768, kernel_size=(32, 32), stride=(32, 32), bias=False)
    (ln_pre): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
    (transformer): Transformer(
      (resblocks): Sequential(
        (0): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          )
          (ln_1): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
          (mlp): Sequential(
            (c_fc): Linear(in_features=768, out_features=3072, bias=True)
            (gelu): QuickGELU()
            (c_proj): Linear(in_features=3072, out_features=768, bias=True)
          )
          (ln_2): LayerNorm((768,), eps=1e-05, elementwise_affine=True)
        )
        (1): ResidualAttentionBlock(
          (attn): MultiheadAttention(
            (out_proj): NonDynamicallyQuantizableLinear(in_features=768, out_features=768, bias=True)
          

In [98]:
image = np.load("./flintstones_data/video_frames_sampled/s_01_e_01_shot_000099_000173.npy")
#from matplotlib import pyplot as plt
#plt.imshow(image, interpolation='nearest')
#plt.show()

In [102]:
image.shape

(5, 128, 128, 3)

In [103]:
from PIL import Image
from numpy import 

TypeError: Cannot handle this data type: (1, 1, 128, 3), |u1